In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
load_dotenv(override = True)
api_key = os.getenv("OPENAI_API_KEY")

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()
    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"
    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("SBIN.BO")

## Analysis of SBIN.BO Stock Data

The data you provided represents the weekly performance of the stock SBIN.BO over a year. Let’s break down the main elements and trends based on this historical data.

### Key Metrics Explained
1. **Open Price**: The stock price at the start of the trading week.
2. **Close Price**: The stock price at the end of the trading week.
3. **High Price**: The highest price reached during the week.
4. **Low Price**: The lowest price reached during the week.
5. **Volume**: The total shares traded during the week.

### Trends Observed
- **Price Movement**: 
  - The starting price several weeks ago ranged approximately between **740** and **800**, showing fluctuations with both upward and downward movements.
  - Recent data points show that the stock price has experienced some volatility, reaching a peak of about **895** and a declining trend down towards the **700s** over the last few weeks.
  
- **Short-Term Trends**: 
  - The stock has shown both upward and downward fluctuations; for instance:
    - The price reached a high of **795.40** at one point in the recent past and has subsequently declined to lower levels around **753** in recent weeks.
    - Noticeably, the stock reached **732.75** before bouncing back, indicating a support level where buyers might be willing to step in.
  
- **Volume Analysis**:
  - The **volume** has varied, with peaks indicating strong buying or selling interest. Notable weeks like **March 3, 2025**, recorded a high trading volume of **2,224,190**, which might imply significant interest or news impacting the stock.

### Total Returns Calculation
To determine the total returns as a percentage over this observation period, we use the formula:

\[
\text{Total Returns (\%)} = \left( \frac{\text{End Value} - \text{Initial Value}}{\text{Initial Value}} \right) \times 100
\]

Assuming the **Initial Value** was around **740** (the weekly open on Jan 6, 2025), and the **End Value** is **753** (the weekly close on March 21, 2025):

\[
\text{Total Returns (\%)} = \left( \frac{753 - 740}{740} \right) \times 100 = \left( \frac{13}{740} \right) \times 100 \approx 1.76\%
\]

### Summary
- **Stability**: The stock has shown periods of price stability with minor fluctuations and periods of volatility. 
- **Trend Direction**: Overall, it has been fluctuating around a central mean price, providing modest returns of approximately **1.76%** over the evaluated weeks.
- **Investment Consideration**: Investors should keep in mind the market conditions, news updates for the stock, and broader economic factors which may impact stock performance going forward.

In conclusion, consistent monitoring of price movements will be crucial for anyone looking to invest in SBIN.BO, keeping a close eye on both the opening and closing prices, as well as volume trends to make informed decisions.

In [9]:
display_analysis("ZOMATO.NS")

# Stock Analysis of ZOMATO.NS

### Overview
ZOMATO.NS is a stock listed on the National Stock Exchange in India. The provided data contains weekly trading information for approximately one year. For each week, we have the opening price, closing price, highest price, lowest price, closing price adjusted for splits/dividends, and trading volume.

### Key Terms
- **Open Price**: The stock price at the beginning of the trading week.
- **Close Price**: The stock price at the end of the trading week.
- **High Price**: The highest price achieved during the week.
- **Low Price**: The lowest price recorded during the week.
- **Adj Close**: The adjusted closing price that accounts for any stock splits and dividends.
- **Volume**: The total number of shares traded during the week.

### Trends Analysis
1. **Price Movement**:
   - The stock shows significant fluctuations over the year.
   - Initially, in January 2024, ZOMATO.NS was trading around ₹274.50. The prices then fluctuated, showing a general downtrend until June 2024 where it dipped down to around ₹179.15.
   - Starting from June 2024, there seems to be a steady rise in the stock price, reaching a high of ₹290.50 in September 2024 before experiencing another downtrend in the following months.

2. **Recent Performance**:
   - The most recent data point shows a closing price of ₹227.52 on March 21, 2025, recovering from a significantly low price before the February peak.
   - Overall, the stock witnessed some of its highest trading volumes during the period of October to December 2024, indicating higher investor interest.

3. **Volatility**:
   - The stock demonstrates a significant level of volatility, with weekly highs and lows indicating considerable price swings. For example, the highest recorded price was around ₹304.70 in December 2024.

### Total Return Calculation
To calculate the total return over the period, we can use the opening price from the first week and the closing price from the most recent week:

- **Opening Price (Jan 6, 2024)**: ₹274.50
- **Closing Price (Mar 21, 2025)**: ₹227.52

The formula for total return in percentage is:
\[
\text{Total Return (\%)} = \left( \frac{\text{Final Value} - \text{Initial Value}}{\text{Initial Value}} \right) \times 100
\]

Calculating the total return:
\[
\text{Total Return} = \left( \frac{227.52 - 274.50}{274.50} \right) \times 100 = \left( \frac{-46.98}{274.50} \right) \times 100 \approx -17.13\%
\]

### Summary
ZOMATO.NS has shown significant price volatility over the past year with notable peaks and valleys. Despite having periods of high prices, the overall return from January 2024 to March 2025 is approximately -17.13%. This indicates that the investment would have decreased in value. Investors should be cautious and consider market conditions before making any decisions on ZOMATO.NS.